# Tweets Featuring

Loading some of the data captured in the previous Jupyter notebook "Read_tweets", let's create some new features.

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%load_ext watermark
%watermark -v -m -p numpy,pandas,pandas_profiling -g

import re
from tqdm import tqdm
import yaml
import watermark
import emoji                      # conda install -c conda-forge emoji
import numpy as np
import pandas as pd
import pandas_profiling
from pandas.plotting import register_matplotlib_converters    # for pandas_profiling

register_matplotlib_converters()                              # for pandas_profiling

CPython 3.7.3
IPython 7.5.0

numpy 1.16.4
pandas 0.24.2

compiler   : GCC 7.3.0
system     : Linux
release    : 5.0.0-20-generic
machine    : x86_64
processor  : x86_64
CPU cores  : 8
interpreter: 64bit
Git hash   : 2d9bde92b9668ab709e38fde6b5d3b06a994a8bd


### Constants

In [2]:
TWEETS_INPUT = "../data/raw/tweets.csv"
PROCESSED_TWEETS = "../data/processed/tweets-processed.csv"
TWITER_USERS_FILE = "../data/raw/twitter_users.csv"
PROCESSED_USERS_FILE = '../data/processed/twitter_users_processed.csv'

### Load Data

In [3]:
tweets_df = pd.read_csv(TWEETS_INPUT, parse_dates=['created_at'])
users_df = pd.read_csv(TWITER_USERS_FILE, parse_dates=['created_at'])

In [4]:
tweets_df.head(3)

,screen_name,location,id,source,coordinates,favorite_count,favorited,lang,hashtags,created_at,text
0,ONYXCONtruth,ATL & The Universe,446395993,Instagram,NaN,0,False,en,"[{'text': 'real', 'indices': [18, 23]}, {'text...",2019-07-04 22:57:55,Salute to all the #real #Artist that make #ONY...
1,CassiniFrank,"Vancouver, British Columbia",997922612255703040,Twitter for iPhone,NaN,0,False,en,"[{'text': 'UK', 'indices': [47, 50]}, {'text':...",2019-07-04 22:57:47,RT @volition_movie: Mighty chuffed to have our...
2,afriwomencinema,NaN,29717530,Twitter Web Client,NaN,0,False,en,"[{'text': 'Senegal', 'indices': [107, 115]}]",2019-07-04 22:57:42,FROM THE AFRICAN WOMEN IN CINEMA BLOG ARCHIVES...


In [5]:
tweets_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 11 columns):
screen_name       4000 non-null object
location          3220 non-null object
id                4000 non-null int64
source            4000 non-null object
coordinates       98 non-null object
favorite_count    4000 non-null int64
favorited         4000 non-null bool
lang              4000 non-null object
hashtags          4000 non-null object
created_at        4000 non-null datetime64[ns]
text              4000 non-null object
dtypes: bool(1), datetime64[ns](1), int64(2), object(7)
memory usage: 316.5+ KB


In [6]:
tweets_df.describe()

,id,favorite_count
count,4.000000e+03,4000.000000
mean,3.263934e+17,0.802750
std,4.615272e+17,4.507707
min,4.046681e+06,0.000000
25%,1.905869e+08,0.000000
50%,1.895170e+09,0.000000
75%,8.439787e+17,0.000000
max,1.147944e+18,131.000000


In [7]:
# tweets_df.profile_report()
pandas_profiling.ProfileReport(tweets_df)

Number of variables,11
Number of observations,4000
Total Missing (%),10.6%
Total size in memory,316.5 KiB
Average record size in memory,81.0 B
Numeric,2
Categorical,6
Boolean,0
Date,1
Text (Unique),0
Rejected,2


From previous information, location has a certain number of NaN which we can replace for the string 'unknown'. <br>
Coordinates on the other side is almost completely null values.

In [8]:
tweets_df['location'].fillna('unknown', inplace=True)

In [9]:
print('Kwnown coordinates: {}%'.format(100 * tweets_df.coordinates.count() / tweets_df.shape[0]))

Kwnown coordinates: 2.45%


In [10]:
tweets_df.drop(['coordinates'], axis=1, inplace=True)

## Extra features: tweets

* Extract tags embeded in the tweet:

In [11]:
tweets_df.text[0]

'Salute to all the #real #Artist that make #ONYXCON POSSIBLE ‼️ #popularArts #Film #ComicBooks #Gaming #HipHop… https://t.co/vFtdbfXvMn'

In [12]:
tag_regex = re.compile(r'#[\w]+')
def get_tags(text, regex=tag_regex):
    tags = regex.findall(text)
    return [k.replace('#', '') for k in tags]

In [13]:
tweets_df['tags'] = tweets_df.text.apply(get_tags, args = (tag_regex,))

In [14]:
def clean_tags(text, tags):
    # Eliminate tags from the text
    for tag in tags:
        text = re.sub(' +', ' ', text.replace('#' + tag, '').replace('\n', ' ') \
                      .replace('\r', '').replace(' ― ', '').replace(' …', ''))
    
    return text.strip()     # Clean up

In [15]:
tweets_df['text'] = tweets_df[['text', 'tags']].apply(lambda x: clean_tags(*x), axis=1)
tweets_df['text'][0]        # Verify updates...

'Salute to all the that make POSSIBLE ‼️ https://t.co/vFtdbfXvMn'

* Retweets

In [16]:
retweet_regex = re.compile(r'RT\s@[\w]+:')
def is_retweet(text, regex=retweet_regex):
    retweet = regex.findall(text)
    is_retweet = False
    author = ''
    if retweet:
        text = text.replace(retweet[0], '').strip()
        is_retweet = True
        author = retweet[0].replace('RT @', '').replace(':', '')
    return is_retweet, author, text

In [17]:
is_retweet, authors, text = zip(*tweets_df.text.apply(is_retweet, args = (retweet_regex,)))

In [18]:
tweets_df['is_retweet'], tweets_df['retweet_author'], tweets_df['text'] = [is_retweet, authors, text]

In [19]:
tweets_df.head(3)

,screen_name,location,id,source,favorite_count,favorited,lang,hashtags,created_at,text,tags,is_retweet,retweet_author
0,ONYXCONtruth,ATL & The Universe,446395993,Instagram,0,False,en,"[{'text': 'real', 'indices': [18, 23]}, {'text...",2019-07-04 22:57:55,Salute to all the that make POSSIBLE ‼️ https:...,"[real, Artist, ONYXCON, popularArts, Film, Com...",False,
1,CassiniFrank,"Vancouver, British Columbia",997922612255703040,Twitter for iPhone,0,False,en,"[{'text': 'UK', 'indices': [47, 50]}, {'text':...",2019-07-04 22:57:47,Mighty chuffed to have our at ’s legendary @Fr...,"[UK, Premiere, London, WestEnd, Fright]",True,volition_movie
2,afriwomencinema,unknown,29717530,Twitter Web Client,0,False,en,"[{'text': 'Senegal', 'indices': [107, 115]}]",2019-07-04 22:57:42,FROM THE AFRICAN WOMEN IN CINEMA BLOG ARCHIVES...,[Senegal],False,


* Number of words in the tweet

In [20]:
tweets_df['n_words'] = tweets_df.text.apply(len)

In [21]:
tweets_df['has_link'] = tweets_df.text.apply(lambda x: 'http' in x)

In [22]:
emoji_regex = emoji.get_emoji_regexp()
def capture_emojis(text):
    emojis = emoji_regex.findall(text)
    if emojis:
        emoji_count = len(emojis)
        for e in emojis:
            text = text.replace(e, '')
        text = text.strip()
        emojis = ' '.join(emojis)
    else:
        emoji_count = 0
        emojis = ''
    
    return emoji_count, emojis, text

In [23]:
emoji_count, emojis, text = zip(*tweets_df.text.apply(capture_emojis))
tweets_df['emoji_count'], tweets_df['emojis'], tweets_df['text'] = [emoji_count, emojis, text]

In [24]:
tweets_df.head(3)

,screen_name,location,id,source,favorite_count,favorited,lang,hashtags,created_at,text,tags,is_retweet,retweet_author,n_words,has_link,emoji_count,emojis
0,ONYXCONtruth,ATL & The Universe,446395993,Instagram,0,False,en,"[{'text': 'real', 'indices': [18, 23]}, {'text...",2019-07-04 22:57:55,Salute to all the that make POSSIBLE ️ https:/...,"[real, Artist, ONYXCON, popularArts, Film, Com...",False,,63,True,1,‼
1,CassiniFrank,"Vancouver, British Columbia",997922612255703040,Twitter for iPhone,0,False,en,"[{'text': 'UK', 'indices': [47, 50]}, {'text':...",2019-07-04 22:57:47,Mighty chuffed to have our at ’s legendary @Fr...,"[UK, Premiere, London, WestEnd, Fright]",True,volition_movie,81,False,0,
2,afriwomencinema,unknown,29717530,Twitter Web Client,0,False,en,"[{'text': 'Senegal', 'indices': [107, 115]}]",2019-07-04 22:57:42,FROM THE AFRICAN WOMEN IN CINEMA BLOG ARCHIVES...,[Senegal],False,,130,True,0,


In [25]:
tweets_df.to_csv(PROCESSED_TWEETS, index=False)

## Extra features: Users Info

In [26]:
users_df.head(3)

,name,screen_name,id,lang,followers_count,location,created_at,statuses_count,friends_count,description
0,mohd akmal,cipanoss,631924707,NaN,3,NaN,2012-07-10 09:43:19,1.0,53.0,b''
1,basset bourouro,bassetbourouro,2184098312,NaN,0,NaN,2013-11-09 10:56:00,2.0,7.0,b''
2,سعود,SAUD9969,597529653,NaN,168,NaN,2012-06-02 14:16:50,11.0,524.0,b'\xd9\x85\xd8\xba\xd8\xb1\xd8\xaf \xd8\xb3\xd...


In [27]:
users_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1582 entries, 0 to 1581
Data columns (total 10 columns):
name               1581 non-null object
screen_name        1582 non-null object
id                 1582 non-null int64
lang               1 non-null float64
followers_count    1582 non-null int64
location           496 non-null object
created_at         1580 non-null datetime64[ns]
statuses_count     1580 non-null float64
friends_count      1580 non-null float64
description        1581 non-null object
dtypes: datetime64[ns](1), float64(3), int64(2), object(4)
memory usage: 123.7+ KB


In [28]:
pandas_profiling.ProfileReport(users_df)

Number of variables,10
Number of observations,1582
Total Missing (%),16.9%
Total size in memory,123.7 KiB
Average record size in memory,80.1 B
Numeric,4
Categorical,3
Boolean,1
Date,1
Text (Unique),1
Rejected,0


Replace null values for friends count with zero:

In [29]:
users_df['friends_count'].fillna(0, inplace=True)

Replace null values at 'created_at' with most common:

In [30]:
users_df['created_at'].fillna(users_df['created_at'].value_counts().index[0], inplace=True)

Fill nan values:

In [31]:
users_df['location'].fillna('unknown', inplace=True)
users_df['name'].fillna('unknown', inplace=True)
users_df['statuses_count'].fillna(0, inplace=True)

In [33]:
users_df.to_csv(PROCESSED_USERS_FILE, index=False)